In [63]:
import torch
import torch.nn as nn
# import tdqm
# import torch.nn.functional as F
# from torch.nn import Parameter

# Dummy model

In [64]:
class ToyNN(nn.Module):
    def __init__(self, input_dim = 100):
        super().__init__()
        self.l1 = nn.Linear(input_dim, input_dim//2)
        self.l2 = nn.Linear(input_dim//2, input_dim//4)
        self.l3 = nn.Linear(input_dim//4, input_dim//8)
        self.l4 = nn.Linear(input_dim//8, 1)
        self.activ = nn.Sigmoid()

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        return self.activ(x)

In [65]:
def get_n_params(model):
    """ Returns the number of learnable parameters of a network
    """
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [66]:
toy = ToyNN(8)
print(f"Number of parameters : {get_n_params(toy)}")
print(toy)

Number of parameters : 51
ToyNN(
  (l1): Linear(in_features=8, out_features=4, bias=True)
  (l2): Linear(in_features=4, out_features=2, bias=True)
  (l3): Linear(in_features=2, out_features=1, bias=True)
  (l4): Linear(in_features=1, out_features=1, bias=True)
  (activ): Sigmoid()
)


In [67]:
for i, layer in enumerate(toy.children()):
    if isinstance(layer, nn.Linear):
        print(f"Layer {i+1} : ")
        print("\t Weight :\n", layer.state_dict()['weight'])
        print("\t Bias :\n", layer.state_dict()['bias'])

Layer 1 : 
	 Weight :
 tensor([[-0.0940, -0.2908,  0.2079, -0.0584, -0.3501,  0.3085,  0.0806,  0.0824],
        [ 0.1025,  0.0392,  0.2947, -0.0558,  0.2762, -0.1001, -0.2862, -0.2062],
        [ 0.1485, -0.3101, -0.1181,  0.2630, -0.1025,  0.2110, -0.3093,  0.1004],
        [ 0.2866, -0.2117, -0.2590, -0.2302, -0.0202,  0.1783,  0.1710, -0.2176]])
	 Bias :
 tensor([ 0.0439,  0.2172,  0.1712, -0.0827])
Layer 2 : 
	 Weight :
 tensor([[ 0.1937, -0.3869, -0.4968, -0.1198],
        [-0.2249,  0.4219, -0.2992, -0.3493]])
	 Bias :
 tensor([-0.1975, -0.4726])
Layer 3 : 
	 Weight :
 tensor([[ 0.5306, -0.0663]])
	 Bias :
 tensor([0.0043])
Layer 4 : 
	 Weight :
 tensor([[0.5173]])
	 Bias :
 tensor([-0.2312])


# ONNX Export

In [68]:
input_example = torch.Tensor([1,1,1,1,1,1,1,1])

torch.onnx.export(
  toy,
  input_example,
  "model.onnx",
  # only needed if you want to specify an opset version > 1.
  custom_opsets={"custom_domain": 2})

# Dummy training

In [74]:
optimizer = torch.optim.Adam(toy.parameters(), lr = 0.0001)

def criterion(x, y):
    return (x - y)**2

def train():
    # toy.zero_grad()
    input_data = torch.randn((8,))

    pred = toy(input_data)
    loss = criterion(pred, 0.7)
    loss.backward()
    optimizer.step()
    return loss.item(), pred.item()

In [77]:
epochs = range(1000)
for e in epochs :
    print(f"{e} : {train()}")


0 : (0.0008561498834751546, tensor([0.7293], grad_fn=<SigmoidBackward0>))
1 : (0.0003579620679374784, tensor([0.6811], grad_fn=<SigmoidBackward0>))
2 : (0.000846973096486181, tensor([0.7291], grad_fn=<SigmoidBackward0>))
3 : (0.0022809673100709915, tensor([0.7478], grad_fn=<SigmoidBackward0>))
4 : (0.0011723536299541593, tensor([0.7342], grad_fn=<SigmoidBackward0>))
5 : (0.00016440756735391915, tensor([0.6872], grad_fn=<SigmoidBackward0>))
6 : (7.67517922213301e-05, tensor([0.7088], grad_fn=<SigmoidBackward0>))
7 : (3.391446080058813e-05, tensor([0.6942], grad_fn=<SigmoidBackward0>))
8 : (0.00047873574658297, tensor([0.7219], grad_fn=<SigmoidBackward0>))
9 : (0.0005768924602307379, tensor([0.7240], grad_fn=<SigmoidBackward0>))
10 : (0.0007128305733203888, tensor([0.7267], grad_fn=<SigmoidBackward0>))
11 : (0.001519004232250154, tensor([0.7390], grad_fn=<SigmoidBackward0>))
12 : (0.0001612678315723315, tensor([0.7127], grad_fn=<SigmoidBackward0>))
13 : (3.311416548967827e-06, tensor([0.

In [71]:
for i, layer in enumerate(toy.children()):
    if isinstance(layer, nn.Linear):
        print(f"Layer {i+1} : ")
        print("\t Weight :\n", layer.state_dict()['weight'])
        print("\t Bias :\n", layer.state_dict()['bias'])

Layer 1 : 
	 Weight :
 tensor([[-0.0816, -0.3013,  0.2188, -0.0456, -0.3384,  0.3196,  0.0688,  0.0728],
        [ 0.0902,  0.0497,  0.2838, -0.0686,  0.2645, -0.1112, -0.2745, -0.1965],
        [ 0.1362, -0.2995, -0.1290,  0.2503, -0.1142,  0.1999, -0.2976,  0.1100],
        [ 0.2742, -0.2011, -0.2699, -0.2430, -0.0319,  0.1672,  0.1828, -0.2079]])
	 Bias :
 tensor([ 0.0565,  0.2046,  0.1586, -0.0953])
Layer 2 : 
	 Weight :
 tensor([[ 0.1892, -0.3752, -0.4845, -0.1325],
        [-0.2201,  0.4101, -0.3117, -0.3366]])
	 Bias :
 tensor([-0.1849, -0.4853])
Layer 3 : 
	 Weight :
 tensor([[ 0.5179, -0.0784]])
	 Bias :
 tensor([0.0169])
Layer 4 : 
	 Weight :
 tensor([[0.5047]])
	 Bias :
 tensor([-0.2185])
